In [220]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import feature
from sklearn.model_selection import train_test_split
import os
from collections import Counter

Manual KNN Classification using LBP

In [221]:
#Convert RGB to Grayscale image
def rgb2greyscale(img):
    return np.dot(img[...,:3], [0.299, 0.587, 0.114])

In [222]:
#Create LBP response
def LBPResponse(image):
    X = [-1, -1, -1, 0, 0, 1, 1, 1]
    Y = [-1, 0, 1, -1, 1, -1, 0, 1]
    kernal = np.array([1, 2, 4, 128, 8, 64, 32, 16])
    new_image = []
    for i in range(1, image.shape[0]-1):
        temp_op = []
        for j in range(1, image.shape[1]-1):
            temp_arr = []
            for k in range(8):
                temp_arr.append(0 if image[i+X[k],j+Y[k]]<image[i,j] else 1)
            temp_op.append(np.dot(kernal, temp_arr))
        new_image.append(temp_op)
    return np.asarray(new_image, dtype=np.uint8)

In [223]:
#Create Manual Histogram
def createHistogram(img):
    output = np.zeros(256, dtype=int)
    for i in img:
        output[i]+=1
    return output

In [224]:
#Process data(Read all images, Convert to Grayscale, Create LBP response, Find histogram, Store with corrresponding labels)
data = []
for item in os.listdir('shapes'):
    for image in os.listdir(f'shapes/{item}'):
        img = plt.imread(f'shapes/{item}/{image}')
        img = rgb2greyscale(img)
        img = LBPResponse(img)
        hist = createHistogram(img)
        data.append([hist, item])
np.random.shuffle(data)

In [225]:
print(data[0])

[array([ 0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  3,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  1,  1,  0,  0, 14,  3,  4,  1,  1,  0,  0,  0,
        0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,
        0,  7,  3,  0,  4,  5,  0,  5,  1,  0,  0,  0,  1,  0,  0,  0, 10,
        0,  0,  0,  0,  0,  0,  0,  4,  0,  0,  0,  1,  0,  0,  0,  5,  0,
        0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,
        0,  0,  4,  0,  6,  0,  0,  2,  9,  0,  0,  0,  6,  0,  0,  0,  0,
        0,  0,  0,  6,  0,  0,  0,  3,  0,  0,  0,  0,  1,  0,  0,  2,  0,
        0,  0,  0,  1,  

In [226]:
#Test Train Split (Test - 5%)
X_train, X_test, Y_train, Y_test = train_test_split(np.asarray([row[0] for row in data]), np.asarray([row[1] for row in data]), test_size=0.05)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(285, 256) (285,) (15, 256) (15,)


In [227]:
#Prediction based on k nearest neighbor
k = 20
accurate = 0
for test_data, value in zip(X_test, Y_test):
    top_k = np.argsort(np.sqrt(np.sum(np.square(X_train-test_data), axis=1)))[0:k]
    result = Counter(Y_train[top_k])
    if(value == max(result, key=result.get)):
        accurate += 1
print(f'Accuracy : {accurate/X_test.shape[0]}')

Accuracy : 0.8666666666666667


KNN Classification with LBP using feature from skimage and numpy histogram

In [228]:
data = []
for item in os.listdir('shapes'):
    for image in os.listdir(f'shapes/{item}'):
        img = plt.imread(f'shapes/{item}/{image}')
        img = rgb2greyscale(img)
        lbp = feature.local_binary_pattern(img, 8, 1, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 8 + 3), range=(0, 8 + 2))
        data.append([hist, item])
np.random.shuffle(data)

In [229]:
print(data[0])

[array([  0,   0,   0,  12,  41,  24,   2,  28,  13, 664]), 'squares']


In [230]:
X_train, X_test, Y_train, Y_test = train_test_split(np.asarray([row[0] for row in data]), np.asarray([row[1] for row in data]), test_size=0.05)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(285, 10) (285,) (15, 10) (15,)


In [231]:
#Prediction based on k nearest neighbor
k = 20
accurate = 0
for test_data, value in zip(X_test, Y_test):
    top_k = np.argsort(np.sqrt(np.sum(np.square(X_train-test_data), axis=1)))[0:k]
    result = Counter(Y_train[top_k])
    if(value == max(result, key=result.get)):
        accurate += 1
print(f'Accuracy : {accurate/X_test.shape[0]}')

Accuracy : 0.8666666666666667
